# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [28]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from sklearn.pipeline import Pipeline
nltk.download(['punkt','wordnet'])
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stopword_list = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
# load data from database
engine = create_engine('sqlite:///messages.db')
con = engine.connect()

# Find the names of tables inside of this database
engine.table_names()

['messages']

In [30]:
df=pd.read_sql('messages',con)
df.head()
X=df['message'].values
Y=df[df.columns[4:]].values.astype(int)

### 2. Write a tokenization function to process your text data

In [31]:
def tokenize(text):
    text=re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    text=nltk.word_tokenize(text)
    lemmatizer= nltk.stem.WordNetLemmatizer()
    clean_tokens=[lemmatizer.lemmatize(i).strip() for i in text if i not in stopword_list]
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [32]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(random_state=0)))
])

array([ 'The system retards the rate at which the water runs off, and water permeates down and recharges the underground aquifer.',
       'This shell is out of gas. #sandy (@Shell Gas Station) [pic]: http://t.co/j9eWplNq',
       'Bad road conditions, worsened in winter by heavy snows, continue to be an impediment to food deliveries, particularly in mountainous areas.',
       ..., "It's a job that I need. Please help me find a job. ",
       'Amid the announcement of a truce, Indian police said suspected militants had killed 11 people late Sunday and early Monday, most of them Hindus -- the majority population in India but the minority in Indian-held Kashmir.',
       'She says she has added ground peanuts for protein and fat, and ground-up leaves from local moringa trees, which are packed with calcium and vitamins.'], dtype=object)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)
pipeline.fit(X_train.astype(str), y_train)
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
from sklearn.metrics import classification_report

for i, col in enumerate(df.columns[5:]):
    print(i,col)
    print(classification_report(y_pred[:,i],y_test[:,i]))

0 request-0
             precision    recall  f1-score   support

          0       0.48      0.63      0.54      1154
          1       0.91      0.85      0.88      5331
          2       0.50      0.33      0.40        69

avg / total       0.83      0.81      0.82      6554

1 offer-0
             precision    recall  f1-score   support

          0       0.98      0.89      0.93      5943
          1       0.44      0.81      0.57       611

avg / total       0.93      0.89      0.90      6554

2 aid_related-0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      6554

3 medical_help-0
             precision    recall  f1-score   support

          0       0.84      0.76      0.80      4245
          1       0.62      0.73      0.67      2309

avg / total       0.76      0.75      0.75      6554

4 medical_products-0
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f03e766eb70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=0, ve

In [36]:
from sklearn.model_selection import GridSearchCV

cv_parameters = parameters = {
              'tfidf__smooth_idf':[True, False],
              #'tfidf__sublinear_tf': [True, False],
              #'tfidf__use_idf': [True, False],
             }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, verbose=5, n_jobs=-1)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] tfidf__smooth_idf=True ..........................................
[CV]  tfidf__smooth_idf=True, score=0.23646071700991608, total=  53.9s
[CV] tfidf__smooth_idf=True ..........................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  tfidf__smooth_idf=True, score=0.24052885837782864, total=  53.9s
[CV] tfidf__smooth_idf=True ..........................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  tfidf__smooth_idf=True, score=0.23118006103763988, total=  53.4s
[CV] tfidf__smooth_idf=True ..........................................


...

[CV] . tfidf__smooth_idf=True, score=0.2451678535096643, total=  53.6s
[CV] tfidf__smooth_idf=True ..........................................


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  4.3min remaining:    0.0s


[CV]  tfidf__smooth_idf=True, score=0.24313326551373346, total=  54.1s
[CV] tfidf__smooth_idf=False .........................................
[CV]  tfidf__smooth_idf=False, score=0.24078311721332316, total=  54.3s
[CV] tfidf__smooth_idf=False .........................................
[CV]  tfidf__smooth_idf=False, score=0.24383422323925757, total=  53.6s
[CV] tfidf__smooth_idf=False .........................................
[CV]  tfidf__smooth_idf=False, score=0.23296032553407936, total=  53.1s
[CV] tfidf__smooth_idf=False .........................................
[CV] . tfidf__smooth_idf=False, score=0.241353001017294, total=  53.2s
[CV] tfidf__smooth_idf=False .........................................
[CV] . tfidf__smooth_idf=False, score=0.237029501525941, total=  54.0s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 10.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
y_pred=cv.predict(X_test)

for i, col in enumerate(df.columns[5:]):
    print(i,col)
    print(classification_report(y_pred[:,i],y_test[:,i]))

0 request-0
             precision    recall  f1-score   support

          0       0.48      0.63      0.54      1154
          1       0.91      0.85      0.88      5331
          2       0.50      0.33      0.40        69

avg / total       0.83      0.81      0.82      6554

1 offer-0
             precision    recall  f1-score   support

          0       0.98      0.89      0.93      5943
          1       0.44      0.81      0.57       611

avg / total       0.93      0.89      0.90      6554

2 aid_related-0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      6554

3 medical_help-0
             precision    recall  f1-score   support

          0       0.84      0.76      0.80      4245
          1       0.62      0.73      0.67      2309

avg / total       0.76      0.75      0.75      6554

4 medical_products-0
           

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.svm import SVC
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def sentence_length(self, text):
        sentence_list = nltk.sent_tokenize(text)
        
        return len(sentence_list)

    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        X_len = pd.Series(X).apply(self.sentence_length)
        return pd.DataFrame(X_len)

In [39]:
new_pipeline = Pipeline([
        ('Feature_Union',FeatureUnion([
            
            ('text_pipeline',Pipeline([
                ('vect',CountVectorizer(tokenizer=tokenize)),
                ('tfid',TfidfTransformer())
            ])),
        
            ('Text_Length',TextLengthExtractor()),
            
        ])),
        
        ('clf',MultiOutputClassifier(RandomForestClassifier(random_state=0)))
    ])



<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('Feature_Union', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])>

In [40]:
parameters = new_parameters = {
              'clf__estimator__n_estimators' : [10,20,50]
              #'tfidf__sublinear_tf': [True, False],
              #'tfidf__use_idf': [True, False],             
             }


In [41]:
cv = GridSearchCV(pipeline, param_grid=new_parameters, cv=5, verbose=5, n_jobs=-1)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] clf__estimator__n_estimators=10 .................................
[CV]  clf__estimator__n_estimators=10, score=0.23646071700991608, total=  54.1s
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, score=0.24052885837782864, total=  53.4s
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, score=0.23118006103763988, total=  53.3s
[CV] clf__estimator__n_estimators=10 .................................


...

[CV]  clf__estimator__n_estimators=10, score=0.2451678535096643, total=  53.4s
[CV] clf__estimator__n_estimators=10 .................................


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  4.2min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, score=0.24313326551373346, total=  54.0s
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.25273328248156623, total= 1.7min
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.2517162471395881, total= 1.7min
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.2489827060020346, total= 1.7min
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.25508646998982704, total= 1.7min
[CV] clf__estimator__n_estimators=20 .................................
[CV] ...... clf__estimator__n_estimators=20, score=0.25, total= 1.7min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.2608695652173913, total= 4.0min
[CV] clf__estimator__n_est

[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 37.2min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [10, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [46]:
pipeline.predict(X_test)

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

### 9. Export your model as a pickle file

In [47]:
import joblib
joblib.dump(pipeline, "optimized_pipline.pkl")

['optimized_pipline.pkl']

In [ ]:
pipeline

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

NameError: name 'cv' is not defined